In [4]:
from libs.reading_utils import get_keywords
from glob import glob
from datetime import datetime

In [5]:
keywords, historicList = get_keywords()


In [6]:
for dic in historicList:
    print(dic)

{'keyword': 'bitcoin OR BTC', 'coinname': 'bitcoin', 'start': datetime.date(2015, 1, 1), 'end': datetime.date(2018, 9, 10)}
{'keyword': 'dashcoin OR DASH OR darkcoin', 'coinname': 'dashcoin', 'start': datetime.date(2015, 5, 1), 'end': datetime.date(2018, 9, 10)}
{'keyword': 'dogecoin OR DOGE', 'coinname': 'dogecoin', 'start': datetime.date(2015, 1, 1), 'end': datetime.date(2018, 9, 10)}
{'keyword': 'ethereum OR ETH', 'coinname': 'ethereum', 'start': datetime.date(2015, 12, 1), 'end': datetime.date(2018, 9, 10)}
{'keyword': 'litecoin OR LTC', 'coinname': 'litecoin', 'start': datetime.date(2015, 1, 1), 'end': datetime.date(2018, 9, 10)}
{'keyword': 'monero OR XMR', 'coinname': 'monero', 'start': datetime.date(2015, 1, 1), 'end': datetime.date(2018, 9, 10)}
{'keyword': 'ripple OR XRP', 'coinname': 'ripple', 'start': datetime.date(2015, 1, 1), 'end': datetime.date(2018, 9, 10)}
{'keyword': 'stellar OR STR', 'coinname': 'stellar', 'start': datetime.date(2015, 1, 1), 'end': datetime.date(201

In [8]:
    def test_get_latest():
        fnames = ['2018-01-01_2018-02-01.csv', '2017-01-01_2017-02-01.csv', '2018-05-01_2018-06-01.csv']
        assert get_latest(fnames) == datetime(2018, 6, 1)
        assert get_latest([]) == None

In [9]:
test_get_latest()

In [2]:
def get_latest(files):
    if (len(files) >= 1):
        endings = [datetime.strptime(file.split("_")[1].replace('.csv', ''), '%Y-%m-%d') for file in files]
        final_date = max(d for d in endings)
        return final_date
    else:
        return None

In [24]:
datetime(2018, 6, 1).date() == datetime(2018, 6, 1).date() 

True

In [21]:
dic

{'keyword': 'stellar OR STR',
 'coinname': 'stellar',
 'start': datetime.date(2015, 1, 1),
 'end': datetime.date(2018, 9, 10)}